In [1]:
using Pkg
Pkg.activate("libs/")
using CSV
using JLD2
using CUDA
using Glob
using Dates
using DICOM
using Images
using ImageView
using ImageDraw
using Statistics
using DataFrames
using StaticArrays
using MLDataPattern
using ChainRulesCore
using FastAI, FastVision, Flux, Metalhead
import CairoMakie; CairoMakie.activate!(type="png")

  Activating project at `c:\Users\wenbl13\OneDrive - UCI Health\Desktop\BAC project\libs`


# Notes

1. In this training, images' pixel values are zoomed into range = [0, 1]. This should not be applied to final plan since different images have different min and max.

2. In this training, only `pixel values` are being used. In future, we can feed more features to the NN to get a better information like `L_mass` or `R_mass`.

# Helper functions

# load data

In [2]:
@load "clean_set_step2.jld2" train_set valid_set

2-element Vector{Symbol}:
 :train_set
 :valid_set

In [4]:
data_dir = raw"C:\BAC_dataset\collected_dataset_for_ML"

"C:\\BAC_dataset\\collected_dataset_for_ML"

## Load train set

In [56]:
num_train_data = size(train_set)[1]
train_set_data = Array{Tuple{Vector{Matrix{Float32}}, Vector{Matrix{Float32}}}}(undef, num_train_data)
for i = 1 : num_train_data
    @views t = train_set[i]
    curr_imgs = Array{Array{Float32,2}}(undef, 4)
    curr_lbls = Array{Array{Float32,2}}(undef, 4)
    for j = 1 : 4 # 4 images each patient
        img_path = t[1][j]
        lbl_path = t[2][j]
        # read dicom images
        img = Float32.(dcm_parse(img_path)[(0x7fe0, 0x0010)])
        # read png images
        lbl = Float32.(Images.load(lbl_path)) .+ 1f0 # class 0 and class 1 ----> class 1 and class 2
        # save 
        curr_imgs[j] = img
        curr_lbls[j] = lbl
    end
    train_set_data[i] = (curr_imgs, curr_lbls)
    break
end

In [57]:
train_set_data

300-element Vector{Tuple{Vector{Matrix{Float32}}, Vector{Matrix{Float32}}}}:
    ([[16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0; … ; 16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0], [16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0; … ; 16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0], [16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0; … ; 16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0], [16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0; … ; 16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0]], [[1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0], [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0], [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0], [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]])
 #

In [58]:
train_set_data[1]

(Matrix{Float32}[[16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0; … ; 16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0], [16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0; … ; 16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0], [16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0; … ; 16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0], [16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0; … ; 16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0]], Matrix{Float32}[[1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0], [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0], [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0], [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]])

In [59]:
train_set_data[1][1]

4-element Vector{Matrix{Float32}}:
 [16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0; … ; 16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0]
 [16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0; … ; 16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0]
 [16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0; … ; 16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0]
 [16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0; … ; 16383.0 16383.0 … 16383.0 16383.0; 16383.0 16383.0 … 16383.0 16383.0]

In [60]:
train_set_data[1][2]

4-element Vector{Matrix{Float32}}:
 [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]
 [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]
 [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]
 [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]

In [43]:
typeof(train_set_data[1])

Tuple{Vector{Matrix{Float32}}, Vector{Matrix{Float32}}}